In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [3]:
from sklearn.linear_model import LinearRegression

In [4]:
from sklearn.linear_model import LogisticRegression

In [5]:
df=pd.read_csv('Weather Data.csv')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date/Time         8784 non-null   object 
 1   Temp_C            8784 non-null   float64
 2   Dew Point Temp_C  8784 non-null   float64
 3   Rel Hum_%         8784 non-null   int64  
 4   Wind Speed_km/h   8784 non-null   int64  
 5   Visibility_km     8784 non-null   float64
 6   Press_kPa         8784 non-null   float64
 7   Weather           8784 non-null   object 
dtypes: float64(4), int64(2), object(2)
memory usage: 549.1+ KB


In [7]:
df.isna().sum()

Date/Time           0
Temp_C              0
Dew Point Temp_C    0
Rel Hum_%           0
Wind Speed_km/h     0
Visibility_km       0
Press_kPa           0
Weather             0
dtype: int64

In [8]:
df.duplicated().sum()

np.int64(0)

In [9]:
df.nunique()

Date/Time           8784
Temp_C               533
Dew Point Temp_C     489
Rel Hum_%             83
Wind Speed_km/h       34
Visibility_km         24
Press_kPa            518
Weather               50
dtype: int64

In [10]:
df.head()

,Date/Time,Temp_C,Dew Point Temp_C,Rel Hum_%,Wind Speed_km/h,Visibility_km,Press_kPa,Weather
0,1/1/2012 0:00,-1.8,-3.9,86,4,8.0,101.24,Fog
1,1/1/2012 1:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2,1/1/2012 2:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog"
3,1/1/2012 3:00,-1.5,-3.2,88,6,4.0,101.27,"Freezing Drizzle,Fog"
4,1/1/2012 4:00,-1.5,-3.3,88,7,4.8,101.23,Fog


In [11]:
df.drop(['Date/Time'],axis=1, inplace=True)

In [12]:
df.Weather.value_counts()

Weather
Mainly Clear                               2106
Mostly Cloudy                              2069
Cloudy                                     1728
Clear                                      1326
Snow                                        390
Rain                                        306
Rain Showers                                188
Fog                                         150
Rain,Fog                                    116
Drizzle,Fog                                  80
Snow Showers                                 60
Drizzle                                      41
Snow,Fog                                     37
Snow,Blowing Snow                            19
Rain,Snow                                    18
Thunderstorms,Rain Showers                   16
Haze                                         16
Drizzle,Snow,Fog                             15
Freezing Rain                                14
Freezing Drizzle,Snow                        11
Freezing Drizzle                

In [13]:
df.Weather=df.Weather.apply(lambda x:'Cloudy' if 'Cloudy' in x else x)

In [14]:
df.Weather=df.Weather.apply(lambda x:'Clear' if  'Clear' in x else x)

In [15]:
df.Weather=df.Weather.apply(lambda x:'Snow' if 'Snow' in x else x)

In [16]:
df.Weather=df.Weather.apply(lambda x:'Rain' if 'Rain' in x else x)

In [17]:
df.Weather=df.Weather.apply(lambda x:'Fog' if 'Fog' in x else x)

In [18]:
df.Weather.value_counts()

Weather
Cloudy                   3797
Clear                    3432
Rain                      662
Snow                      583
Fog                       241
Drizzle                    41
Haze                       16
Freezing Drizzle            7
Freezing Drizzle,Haze       3
Thunderstorms               2
Name: count, dtype: int64

In [19]:
df.Weather=df.Weather.apply(lambda x:'others' if x in['Drizzle','Haze','Freezing Drizzle','Freezing Drizzle,Haze','Thunderstorms'] else x)

In [20]:
df.Weather.value_counts()

Weather
Cloudy    3797
Clear     3432
Rain       662
Snow       583
Fog        241
others      69
Name: count, dtype: int64

In [21]:
df.head(1)

,Temp_C,Dew Point Temp_C,Rel Hum_%,Wind Speed_km/h,Visibility_km,Press_kPa,Weather
0,-1.8,-3.9,86,4,8.0,101.24,Fog


In [22]:
df.Weather=df['Weather'].apply(
    lambda x:1 if x=='Cloudy'
    else 2 if x=='Clear'
    else 3 if x=='Rain'
    else 4 if x=='Snow'
    else 5 if x=='Fog'
    else 6
)

In [23]:
df.Weather.unique()

array([5, 1, 3, 2, 4, 6])

In [24]:
df.head(1)

,Temp_C,Dew Point Temp_C,Rel Hum_%,Wind Speed_km/h,Visibility_km,Press_kPa,Weather
0,-1.8,-3.9,86,4,8.0,101.24,5


In [25]:
X=df.drop(['Weather'], axis=1)

In [26]:
y=df.Weather

In [27]:
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=.2, random_state=6464)

In [28]:
X_train.head(1)

,Temp_C,Dew Point Temp_C,Rel Hum_%,Wind Speed_km/h,Visibility_km,Press_kPa
6862,0.0,-4.7,71,9,25.0,102.47


In [29]:
trf1=ColumnTransformer([
    ('imputed_Columns',SimpleImputer(),[0,5])
],remainder='passthrough')

In [30]:
trf2=ColumnTransformer([('Scale',MinMaxScaler(),slice(0,10))],remainder='passthrough')

In [31]:
trf3=DecisionTreeClassifier()

In [32]:
pipe=Pipeline([
    ('imputer', trf1),
    ('scaler', trf2),
    ('model', trf3)
])

In [33]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('imputer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imputed_Columns',
                                                  SimpleImputer(), [0, 5])])),
                ('scaler',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('model', DecisionTreeClassifier())])

In [34]:
y_pred_train=pipe.predict(X_train)

In [35]:
y_pred_test=pipe.predict(X_test)

In [36]:
from sklearn.metrics import accuracy_score

In [37]:
accuracy_score(y_pred_test,y_test)

0.6021627774615822

In [38]:
accuracy_score(y_pred_train,y_train)

1.0

In [39]:
pipe_2=Pipeline([
    ('imputer',trf1),
    ('Scle',trf2),
    ('model',LogisticRegression())
])

In [40]:
pipe_2.fit(X_train,y_train)

Pipeline(steps=[('imputer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imputed_Columns',
                                                  SimpleImputer(), [0, 5])])),
                ('Scle',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('model', LogisticRegression())])

In [41]:
y_pred_test=pipe_2.predict(X_test)

In [42]:
y_pred_train=pipe_2.predict(X_train)

In [43]:
accuracy_score(y_pred_train,y_train)

0.6052369432190123

In [44]:
accuracy_score(y_pred_test,y_test)

0.6067159931701764

In [45]:
from sklearn.metrics import classification_report

In [46]:
print(classification_report(y_pred_test,y_test))

              precision    recall  f1-score   support

           1       0.65      0.58      0.61       853
           2       0.60      0.64      0.62       678
           3       0.33      0.48      0.39        75
           4       0.71      0.70      0.71       114
           5       0.53      0.76      0.62        37
           6       0.00      0.00      0.00         0

    accuracy                           0.61      1757
   macro avg       0.47      0.52      0.49      1757
weighted avg       0.62      0.61      0.61      1757



C:\Users\light tech\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\light tech\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\light tech\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [47]:
print(classification_report(y_train,y_pred_train))

              precision    recall  f1-score   support

           1       0.58      0.65      0.61      3042
           2       0.62      0.61      0.61      2716
           3       0.61      0.34      0.44       552
           4       0.70      0.67      0.68       471
           5       0.66      0.57      0.61       188
           6       0.00      0.00      0.00        58

    accuracy                           0.61      7027
   macro avg       0.53      0.47      0.49      7027
weighted avg       0.60      0.61      0.60      7027



C:\Users\light tech\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\light tech\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\light tech\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [48]:
pipe_3=Pipeline([
    ('imputer',trf1),
    ('scaler',trf2),
    ('model',LinearRegression())
])

In [49]:
pipe_3.fit(X_train,y_train)

Pipeline(steps=[('imputer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imputed_Columns',
                                                  SimpleImputer(), [0, 5])])),
                ('scaler',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('Scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('model', LinearRegression())])

In [50]:
y_pred_test=pipe_3.predict(X_test)

In [51]:
y_pred_train=pipe_3.predict(X_train)

In [52]:
from sklearn.metrics import r2_score

In [53]:
r2_score(y_pred_test,y_test)

-1.5884116144172538

In [54]:
r2_score(y_pred_train,y_train)

-1.6662325940838651